<a href="https://colab.research.google.com/github/MariaChuquitarco/014MBID/blob/main/TFM_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proceso KDD


##Obtención de la base de datos